Experiment,Variable to Change,Fixed Variables,Purpose
1. Sensitivity,"Threshold % (0, 1, 5, 10, 20)",Model: 1.5B  Method: NF4,Find the optimal trade-off (Sweet Spot).
2. Methods,"Method (NF4, AWQ, GPTQ)",Model: 1.5B  Threshold: 5% (or Sweet Spot),Compare KLD impact on different quantizers.
3. Scaling,"Model Size (1.5B, 7B)",Method: (Best of Exp 2)  Threshold: 5%,Test if larger models behave differently.

# **Setup & Dependencies**

**We use L4 GPU**

In [1]:
!pip uninstall transformers torch torchaudio torchvision wandb -y
!pip install llmcompressor
!pip install -q accelerate bitsandbytes datasets scipy matplotlib wandb

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset, concatenate_datasets
from datasets import Dataset
import copy
import gc
import time
from tqdm import tqdm
import shutil
import wandb

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: wandb 0.23.1
Uninstalling wandb-0.23.1:
  Successfully uninstalled wandb-0.23.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.2/273.2 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Set for reproducibility
import random
import numpy as np
from transformers import set_seed

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
set_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## **Experiment 2 Configuration**

In [3]:
# --- Experiment Settings ---
MODELS_TO_TEST = ["Qwen/Qwen2.5-1.5B-Instruct"]
SENSITIVITY_THRESHOLDS = [0.0, 0.05, 0.1, 0.2, 0.3]
CALIBRATION_SAMPLES = 128
EVAL_SAMPLES = 5000
WANDB_PROJECT_NAME = "KLD_Quantization_Exp2"

In [4]:
import wandb
import pandas as pd
from datasets import load_dataset, concatenate_datasets
import os
os.environ["WANDB_QUIET"] = "true"

wandb.login()

if 'results_table' not in globals():
    results_table = []

print("Loading MMLU Dataset...")
try:
    mmlu_dataset = concatenate_datasets([
        load_dataset("cais/mmlu", "all", split='test')
    ])
    print(f"MMLU Dataset Loaded. Size: {len(mmlu_dataset)} samples.")
except Exception as e:
    print(f"Error loading MMLU: {e}")
    from datasets import Dataset
    mmlu_dataset = Dataset.from_dict({
        "question": ["1+1=?"], "choices": [["1", "2", "3", "4"]], "answer": [1]
    })

print("Global setup complete. Ready for Step 2.")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jd4123 (yq171014-columbia-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

Generating test split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1531 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/285 [00:00<?, ? examples/s]

Generating auxiliary_train split:   0%|          | 0/99842 [00:00<?, ? examples/s]

MMLU Dataset Loaded. Size: 14042 samples.
Global setup complete. Ready for Step 2.


## **Metrics & Helper Functions**

In [5]:
def recursive_getattr(obj, attr):
    for part in attr.split('.'):
        obj = getattr(obj, part)
    return obj

def recursive_setattr(obj, attr, val):
    pre, _, post = attr.rpartition('.')
    parent = recursive_getattr(obj, pre) if pre else obj
    setattr(parent, post, val)

In [6]:
# --- MMLU Logic ---
def format_mmlu_prompt(example):
    options = [f"{label}. {example['choices'][i]}" for i, label in enumerate(['A', 'B', 'C', 'D'])]
    prompt_text = f"Question: {example['question']}\nOptions:\n" + "\n".join(options) + "\nAnswer:"
    messages = [
        {"role": "system", "content": "Output only the single letter (A, B, C, or D) corresponding to the correct answer."},
        {"role": "user", "content": prompt_text}
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

def get_mmlu_predictions(model, dataset, num_samples):
    predictions, ground_truths = [], []
    choices = ["A", "B", "C", "D"]
    choice_ids = [tokenizer.encode(c)[0] for c in choices]

    for i in tqdm(range(min(num_samples, len(dataset))), desc="MMLU Eval"):
        ex = dataset[i]
        inputs = tokenizer(format_mmlu_prompt(ex), return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits[0, -1, choice_ids]
            pred = choices[torch.argmax(logits).item()]
        predictions.append(pred)
        ground_truths.append(choices[ex['answer']])
    return predictions, ground_truths

In [7]:
# --- Metrics Helpers ---
def compute_kld(logits_p, logits_q):
    p_probs = F.softmax(logits_p, dim=-1)
    q_log_probs = F.log_softmax(logits_q, dim=-1)
    return nn.KLDivLoss(reduction='batchmean')(q_log_probs, p_probs).item()

def calculate_flip_rate(base_preds, new_preds):
    """Calculates % of answers that changed from the baseline."""
    if not base_preds or not new_preds: return 0.0
    flips = sum([1 for b, n in zip(base_preds, new_preds) if b != n])
    return flips / len(base_preds)

def compute_perplexity(model, tokenizer):
    """Computes perplexity on a subset of WikiText-2"""
    encodings = tokenizer("\n\n".join(load_dataset("wikitext", "wikitext-2-raw-v1", split="test")["text"][:20]), return_tensors="pt")
    max_length = model.config.max_position_embeddings
    stride = 512
    seq_len = encodings.input_ids.size(1)

    nlls = []
    prev_end_loc = 0
    for begin_loc in tqdm(range(0, seq_len, stride), desc="Computing PPL"):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            nlls.append(outputs.loss)

        prev_end_loc = end_loc
        if end_loc == seq_len: break

    return torch.exp(torch.stack(nlls).mean()).item()

def measure_efficiency(model, tokenizer, input_text="Hello world"):
    # 1. Cleanup
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.synchronize()

    # 2. Measure Static Memory (Model Weights Only)
    # This shows the pure effect of quantization storage
    static_mem_bytes = torch.cuda.memory_allocated()
    static_mem_gb = static_mem_bytes / 1024**3

    # 3. Run Inference
    input_ids = tokenizer(input_text, return_tensors="pt").to(device)
    start_time = time.time()
    with torch.no_grad():
        _ = model.generate(**input_ids, max_new_tokens=50, min_new_tokens=50)
    torch.cuda.synchronize()
    end_time = time.time()

    # 4. Measure Peak Memory (Weights + KV Cache + Temp Buffers)
    # This shows the "True Cost" to run the model
    peak_mem_bytes = torch.cuda.max_memory_allocated()
    peak_mem_gb = peak_mem_bytes / 1024**3

    latency = end_time - start_time

    return latency, static_mem_gb, peak_mem_gb

def evaluate_full_suite(model, tokenizer, dataset, metric_name):
    """Runs all metrics and returns them."""
    print(f"--- Evaluating: {metric_name} ---")

    # 1. Accuracy
    preds, truths = get_mmlu_predictions(model, dataset, EVAL_SAMPLES)
    acc = sum([1 for p, g in zip(preds, truths) if p == g]) / len(truths)

    # 2. Perplexity
    ppl = compute_perplexity(model, tokenizer)

    # 3. Efficiency (Unpack 3 values now)
    lat, static_mem, peak_mem = measure_efficiency(model, tokenizer)

    print(f"Results -> Acc: {acc:.2%}, PPL: {ppl:.2f}, Latency: {lat:.2f}s, Static Mem: {static_mem:.2f}GB, Peak Mem: {peak_mem:.2f}GB")

    # Return separate memory metrics
    return acc, ppl, lat, static_mem, peak_mem, preds

## **Advanced Sensitivity Profiling**

In [8]:
def profile_restoration_sensitivity(model_q, model_ref, calib_input, granularity='layer'):
    """
    Profiles sensitivity by measuring the KLD improvement when restoring
    individual parts of the quantized model (model_q) back to FP16 (model_ref).

    Returns:
        sensitivity_scores: Dict mapping name -> KLD improvement (Higher is more sensitive).
    """
    print(f"Profiling Restoration Sensitivity (Granularity: {granularity})...")

    # Compute Baseline
    model_ref.eval()

    with torch.no_grad():
        ref_device = next(model_ref.parameters()).device
        base_logits = model_ref(calib_input.to(ref_device)).logits.to(device)
        current_logits = model_q(calib_input.to(device)).logits
        initial_kld = compute_kld(base_logits, current_logits)

    print(f"Initial Quantized KLD: {initial_kld:.6f}")

    sensitivity_scores = {}

    def get_module_by_name(module, access_string):
        names = access_string.split(sep='.')
        return reduce(getattr, names, module)

    from functools import reduce

    # Block-wise or Layer-wise
    if granularity == 'block':
        if hasattr(model_q, 'model') and hasattr(model_q.model, 'layers'):
            iterable_items = list(enumerate(model_q.model.layers))
            prefix = "model.model.layers"
        else:
            raise ValueError("Could not detect transformer blocks structure.")
        iterator = tqdm(iterable_items, desc="Profiling Blocks")
    elif granularity == 'layer':
        # # We limit this to just the linear layers to save time
        # iterable_items = [(n, m) for n, m in model_q.named_modules() if isinstance(m, (nn.Linear,  import_bnb_linear_type_if_needed()))]
        iterable_items = [(n, m) for n, m in model_q.named_modules()
                          if "mlp" in n or "self_attn" in n]
        iterator = tqdm(iterable_items, desc="Profiling Layers")

    # Restoration Loop
    for name_or_idx, module_q in iterator:
        target_name = f"{prefix}.{name_or_idx}" if granularity == 'block' else name_or_idx
        try:
            module_ref = recursive_getattr(model_ref, target_name)
            backup_quant_module = recursive_getattr(model_q, target_name)
            module_fp16_gpu = copy.deepcopy(module_ref).to(device)
            recursive_setattr(model_q, target_name, module_fp16_gpu)

            # Measure New KLD
            with torch.no_grad():
                new_logits = model_q(calib_input.to(device)).logits
                new_kld = compute_kld(base_logits, new_logits)

            improvement = initial_kld - new_kld
            sensitivity_scores[target_name] = improvement
            recursive_setattr(model_q, target_name, backup_quant_module)

            # Cleanup VRAM
            del module_fp16_gpu

        except Exception as e:
            print(f"Skipping {target_name}: {e}")

    return sensitivity_scores

## **The "Surgery" Implementation**

In [17]:
def perform_surgery(model, sensitive_names, fp16_model_cpu):
    """
    Replaces the sensitive quantized layers in 'model' (GPU)
    with the original FP16 layers from 'fp16_model_cpu' (CPU).

    This Generic Version uses deepcopy, so it works for:
    - Individual Linear layers (gate_proj, q_proj)
    - Entire Blocks (Qwen2MLP, Qwen2Attention)
    """
    count = 0
    print(f"Surgery: Replacing {len(sensitive_names)} Sensitive Layers with FP16...")

    for name in sensitive_names:
        try:
            # 1. Get original FP16 module from CPU backup
            #    (This handles Linear, Qwen2MLP, Qwen2Attention, etc.)
            original_module = recursive_getattr(fp16_model_cpu, name)

            # 2. Create a deep copy and move to GPU
            #    We use deepcopy instead of manually instantiating nn.Linear.
            #    This preserves the exact class type and configuration.
            module_fp16_gpu = copy.deepcopy(original_module).to(model.device)

            # 3. Swap into the quantized model
            recursive_setattr(model, name, module_fp16_gpu)

            count += 1

        except Exception as e:
            print(f"Skipping layer {name}: {e}")

    print(f"Surgery Complete: {count} layers restored.")

# Model Selection & Baseline Evaluation

In [10]:
# Select model
CURRENT_MODEL_ID = MODELS_TO_TEST[0]

print(f"{'='*40}\nSelected Model: {CURRENT_MODEL_ID}\n{'='*40}")

tokenizer = AutoTokenizer.from_pretrained(CURRENT_MODEL_ID)
print("Loading FP16 Baseline (This may take a minute)...")
model_fp16 = AutoModelForCausalLM.from_pretrained(
    CURRENT_MODEL_ID,
    dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

# Evaluate Baseline
base_acc, base_ppl, base_lat, base_static_mem, base_peak_mem, base_preds = evaluate_full_suite(
    model_fp16, tokenizer, mmlu_dataset, "FP16 Baseline"
)

# Log Baseline to WandB
run = wandb.init(project=WANDB_PROJECT_NAME, name=f"{CURRENT_MODEL_ID.split('/')[-1]}-Baseline", reinit=True)
wandb.log({
    "Accuracy": base_acc,
    "Perplexity": base_ppl,
    "Latency": base_lat,
    "Static_Memory": base_static_mem,
    "Peak_Memory": base_peak_mem,
    "Threshold": 0,
    "Flip_Rate": 0.0,
    "Method": "Baseline"
})
run.finish()

# Store in Results Table
results_table.append({
    "Model": CURRENT_MODEL_ID,
    "Method": "FP16 Baseline",
    "Threshold": 0,
    "Acc": base_acc,
    "Flip": 0.0,
    "PPL": base_ppl,
    "Latency": base_lat,
    "Static Mem": base_static_mem,
    "Peak Mem": base_peak_mem
})

print("Baseline Loaded & Evaluated.")

Selected Model: Qwen/Qwen2.5-1.5B-Instruct


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Computing PPL:   0%|          | 0/3 [00:00<?, ?it/s]


Results -> Acc: 57.04%, PPL: 6.33, Latency: 1.91s, Static Mem: 2.88GB, Peak Mem: 2.89GB


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Baseline Loaded & Evaluated.


In [11]:
# Profiling & Offloading
print("Preparing Calibration Data...")
calib_data = tokenizer(
    "\n\n".join(load_dataset("wikitext", "wikitext-2-raw-v1", split="test")["text"][:10]),
    return_tensors="pt"
).input_ids.to(device)

granularity_mode = 'layer'

# Offload FP16 Model to CPU to save memory
print("Moving FP16 model to CPU to free up VRAM...")
model_fp16.cpu()
torch.cuda.empty_cache()
print("VRAM Cleared. Ready for Experiments.")

Preparing Calibration Data...
Moving FP16 model to CPU to free up VRAM...
VRAM Cleared. Ready for Experiments.


# Preliminary Check:
Justifying the "Base" Precision
Variable: Floating Point Type (FP8 vs. FP4/NF4)

Purpose: Before running complex KLD experiments, you must decide what your "base" low-precision format is. If FP4 destroys the model completely and FP8 is perfect, then KLD is needed for FP4 but not FP8. If both are good, FP4 is better for efficiency.

Design:

Run: 1.5B Model (FP16 Baseline) vs. 1.5B (FP8) vs. 1.5B (NF4).

Metric: Perplexity & MMLU Accuracy.

Hypothesis: NF4 offers higher compression but higher degradation than FP8. This justifies using NF4 (or INT4 methods) as the primary candidate for your KLD restoration because it needs the help more than FP8 does.

Decision: If verified, fix 4-bit (NF4/INT4) as the standard base for the rest of the experiments.

**Refer to the FP8 vs FP4 notebook**

**Results: NF4 offers higher compression but higher degradation than FP8. We will use NF4 (or INT4 methods) as the primary candidate for the following experiments because it needs the help more than FP8 does.**

# Experiment 1: Sensitivity Analysis
Research Question: How much of the model actually needs to be kept in high precision to recover performance? Is there a point of diminishing returns?

Fixed Variables:

Model: Qwen2.5-1.5B (Small enough to run fast, big enough to show signal).

Method: NF4 (The simplest 4-bit baseline).

Independent Variable (Change this):

KLD Threshold / % Restored: 0% (Baseline), 1%, 5%, 10%, 20%.


**Refer to the FP8 vs FP4 notebook**


# Experiment 2: Algorithm Comparison
Research Question: Does KLD guidance work better on top of simple quantization (NF4) or advanced quantization (AWQ/GPTQ)?

Fixed Variables:

Model: Qwen2.5-1.5B (Consistent with Exp 1).

Threshold: Fix this to the "winner" from Exp 1 (e.g., if 5% was the sweet spot, use 5% for all).

Independent Variable (Change this):

Method: NF4 vs. AWQ vs. GPTQ.

Why: AWQ and GPTQ already do some optimization. You want to see if your KLD method adds value on top of them, or if it's only useful for naive methods like NF4.

#### AWQ

In [12]:
from llmcompressor.modifiers.awq import AWQModifier
from llmcompressor.modifiers.quantization import GPTQModifier
from llmcompressor import oneshot

In [13]:
# AWQ
print(f"\n--- Starting Experiment: AWQ ({CURRENT_MODEL_ID}) ---")

print("Running AWQ Oneshot Quantization...")
ds = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
calib_data_obj = Dataset.from_dict({"text": [text for text in ds["text"] if len(text) > 0][:128]})

recipe = [AWQModifier(targets="Linear", scheme="W4A16")]
oneshot(
    model=CURRENT_MODEL_ID,
    dataset=calib_data_obj,
    recipe=recipe,
    output_dir="./awq_temp",
    num_calibration_samples=128,
    max_seq_length=512,
    save_compressed=True
)

model_awq = AutoModelForCausalLM.from_pretrained(
    "./awq_temp", device_map="auto", trust_remote_code=True
)

print("Profiling AWQ Sensitivity...")
awq_sensitivity = profile_restoration_sensitivity(
    model_q=model_awq,
    model_ref=model_fp16,
    calib_input=calib_data,
    granularity='block'
)

sorted_awq = sorted(awq_sensitivity.items(), key=lambda x: x[1], reverse=True)
all_layer_names = [n for n, s in sorted_awq]

# Experiment loop
sorted_thresholds = sorted(SENSITIVITY_THRESHOLDS)
current_restored_count = 0

for threshold in sorted_thresholds:
    print(f"\nTargeting Threshold: {threshold:.0%} kept in FP16")

    target_count = int(len(all_layer_names) * threshold)
    layers_to_fix_now = all_layer_names[current_restored_count : target_count]

    if layers_to_fix_now:
        print(f"Restoring {len(layers_to_fix_now)} additional layers...")
        perform_surgery(model_awq, layers_to_fix_now, model_fp16)
        current_restored_count = target_count

    run = wandb.init(
        project=WANDB_PROJECT_NAME,
        name=f"{CURRENT_MODEL_ID.split('/')[-1]}-AWQ-{threshold}",
        config={"model": CURRENT_MODEL_ID, "method": "KLD-AWQ", "threshold": threshold},
        reinit=True
    )

    acc, ppl, lat, static_mem, peak_mem, preds = evaluate_full_suite(
        model_awq, tokenizer, mmlu_dataset, f"KLD-AWQ-{threshold:.0%}"
    )

    flip = calculate_flip_rate(base_preds, preds)

    wandb.log({
        "Accuracy": acc, "Perplexity": ppl, "Latency": lat, "Static_Memory": static_mem,
        "Peak_Memory": peak_mem, "Flip_Rate": flip, "Threshold": threshold
    })

    results_table.append({
        "Model": CURRENT_MODEL_ID,
        "Method": "KLD-AWQ",
        "Threshold": threshold,
        "Acc": acc,
        "Flip": flip,
        "PPL": ppl,
        "Latency": lat,
        "Static Mem": static_mem,
        "Peak Mem": peak_mem
    })
    run.finish()

shutil.rmtree("./awq_temp")
del model_awq
torch.cuda.empty_cache()


--- Starting Experiment: AWQ (Qwen/Qwen2.5-1.5B-Instruct) ---
Running AWQ Oneshot Quantization...


`torch_dtype` is deprecated! Use `dtype` instead!


Tokenizing:   0%|          | 0/128 [00:00<?, ? examples/s]

2025-12-17T02:17:59.510724+0000 | reset | INFO - Compression lifecycle reset
2025-12-17T02:17:59.520085+0000 | _create_default_logger | INFO - Logging all LLM Compressor modifier-level logs to sparse_logs/17-12-2025_02.17.59.log
2025-12-17T02:17:59.521046+0000 | from_modifiers | INFO - Creating recipe from modifiers
2025-12-17T02:17:59.555096+0000 | on_initialize | INFO - No AWQModifier.mappings provided, inferring from model...


Resolving mapping 1/4 (0 skipped): : 28it [00:00, 1268.16it/s]
Resolving mapping 2/4 (27 skipped): : 28it [00:00, 1547.49it/s]
Resolving mapping 3/4 (0 skipped): : 28it [00:00, 1228.78it/s]
Resolving mapping 4/4 (0 skipped): : 28it [00:00, 1561.46it/s]

...

Results -> Acc: 51.90%, PPL: 7.77, Latency: 8.93s, Static Mem: 4.06GB, Peak Mem: 6.24GB



Targeting Threshold: 30% kept in FP16


--- Evaluating: KLD-AWQ-30% ---


Computing PPL:   0%|          | 0/3 [00:00<?, ?it/s]


Results -> Acc: 51.90%, PPL: 7.77, Latency: 9.09s, Static Mem: 4.06GB, Peak Mem: 6.24GB


#### GPTQ

In [18]:
# GPTQ

print(f"\n--- Starting Experiment: GPTQ ({CURRENT_MODEL_ID}) ---")

print("Running GPTQ Optimization...")
ds = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
calib_data_obj = Dataset.from_dict({"text": [text for text in ds["text"] if len(text) > 0][:128]})

recipe = [
    GPTQModifier(
        targets="Linear",
        scheme="W4A16",
        ignore=["lm_head"],
        dampening_frac=0.01
    )
]

oneshot(
    model=CURRENT_MODEL_ID,
    dataset=calib_data_obj,
    recipe=recipe,
    output_dir="./gptq_temp",
    num_calibration_samples=128,
    max_seq_length=512,
    save_compressed=True
)

model_gptq = AutoModelForCausalLM.from_pretrained(
    "./gptq_temp", device_map="auto", trust_remote_code=True, torch_dtype=torch.bfloat16
)

print("Profiling GPTQ Sensitivity...")
gptq_sensitivity = profile_restoration_sensitivity(
    model_q=model_gptq,
    model_ref=model_fp16,
    calib_input=calib_data,
    granularity= 'layer'
)

sorted_gptq = sorted(gptq_sensitivity.items(), key=lambda x: x[1], reverse=True)
all_layer_names = [n for n, s in sorted_gptq]

sorted_thresholds = sorted(SENSITIVITY_THRESHOLDS)
current_restored_count = 0

for threshold in sorted_thresholds:
    print(f"\nTargeting Threshold: {threshold:.0%} kept in FP16")

    target_count = int(len(all_layer_names) * threshold)
    layers_to_fix_now = all_layer_names[current_restored_count : target_count]

    if layers_to_fix_now:
        print(f"Restoring {len(layers_to_fix_now)} additional layers...")
        perform_surgery(model_gptq, layers_to_fix_now, model_fp16)
        current_restored_count = target_count

    run = wandb.init(
        project=WANDB_PROJECT_NAME,
        name=f"{CURRENT_MODEL_ID.split('/')[-1]}-GPTQ-{threshold}",
        config={"model": CURRENT_MODEL_ID, "method": "KLD-GPTQ", "threshold": threshold},
        reinit=True
    )

    acc, ppl, lat, mem, preds = evaluate_full_suite(
        model_gptq, tokenizer, mmlu_dataset, f"KLD-GPTQ-{threshold:.0%}"
    )

    flip = calculate_flip_rate(base_preds, preds)

    wandb.log({
        "Accuracy": acc, "Perplexity": ppl, "Latency": lat, "Static_Memory": static_mem,
        "Peak_Memory": peak_mem, "Flip_Rate": flip, "Threshold": threshold
    })

    results_table.append({
        "Model": CURRENT_MODEL_ID,
        "Method": "KLD-GPTQ",
        "Threshold": threshold,
        "Acc": acc,
        "Flip": flip,
        "PPL": ppl,
        "Latency": lat,
        "Static Mem": static_mem,
        "Peak Mem": peak_mem
    })
    run.finish()

shutil.rmtree("./gptq_temp")
del model_gptq
torch.cuda.empty_cache()


--- Starting Experiment: GPTQ (Qwen/Qwen2.5-1.5B-Instruct) ---
Running GPTQ Optimization...


Tokenizing:   0%|          | 0/128 [00:00<?, ? examples/s]

2025-12-17T04:18:47.673653+0000 | reset | INFO - Compression lifecycle reset
2025-12-17T04:18:47.677213+0000 | from_modifiers | INFO - Creating recipe from modifiers
2025-12-17T04:18:47.716021+0000 | initialize | INFO - Compression lifecycle initialized for 1 modifiers
2025-12-17T04:18:47.717778+0000 | IndependentPipeline | INFO - Inferred `SequentialPipeline` for `GPTQModifier`


(1/29): Calibrating: 100%|██████████| 128/128 [00:02<00:00, 45.88it/s]

2025-12-17T04:18:52.836161+0000 | compress_modules | INFO - Quantizing model.layers.0.self_attn.q_proj using 128 samples


...

Profiling Layers: 100%|██████████| 280/280 [00:12<00:00, 22.97it/s] 

Skipping model.layers.24.mlp: CUDA out of memory. Tried to allocate 28.00 MiB. GPU 0 has a total capacity of 22.16 GiB of which 1.38 MiB is free. Process 27680 has 22.15 GiB memory in use. Of the allocated memory 21.51 GiB is allocated by PyTorch, and 379.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Skipping model.layers.24.mlp.gate_proj: CUDA out of memory. Tried to allocate 28.00 MiB. GPU 0 has a total capacity of 22.16 GiB of which 1.38 MiB is free. Process 27680 has 22.15 GiB memory in use. Of the allocated memory 21.51 GiB is allocated by PyTorch, and 379.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documen

--- Evaluating: KLD-GPTQ-0% ---


MMLU Eval:  14%|█▍        | 698/5000 [01:28<09:05,  7.88it/s]


KeyboardInterrupt: 

### Visualization

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Create DataFrame
df = pd.DataFrame(results_table)

# Display the data
print(df)

# 2. Set up the figure with 2 subplots (Static vs Peak)
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# --- Plot A: Static Memory (Model Size) vs Flip Rate ---
sns.scatterplot(
    data=df, x='Static Mem', y='Flip', hue='Method', style='Method',
    s=200, palette='viridis', ax=axes[0]
)
axes[0].set_title("Compression Efficiency: Static Memory vs Flip Rate")
axes[0].set_xlabel("Static Memory (GB) - [Weights Only]")
axes[0].set_ylabel("Flip Rate (Lower is Better)")
axes[0].grid(True, alpha=0.3)

# Add labels for Plot A
for i in range(df.shape[0]):
    row = df.iloc[i]
    # Check if 'Static_Mem' exists to avoid errors
    if 'Static_Mem' in row:
        axes[0].text(row.Static_Mem + 0.01, row.Flip + 0.001, f"{row.Threshold:.0%}", fontsize=9)

# --- Plot B: Peak Memory (Runtime Cost) vs Flip Rate ---
sns.scatterplot(
    data=df, x='Peak Mem', y='Flip', hue='Method', style='Method',
    s=200, palette='magma', ax=axes[1]
)
axes[1].set_title("Runtime Efficiency: Peak Memory vs Flip Rate")
axes[1].set_xlabel("Peak Memory (GB) - [Weights + Activations + Overhead]")
axes[1].set_ylabel("Flip Rate (Lower is Better)")
axes[1].grid(True, alpha=0.3)

# Add labels for Plot B
for i in range(df.shape[0]):
    row = df.iloc[i]
    if 'Peak_Mem' in row:
        axes[1].text(row.Peak_Mem + 0.05, row.Flip + 0.001, f"{row.Threshold:.0%}", fontsize=9)

plt.tight_layout()
plt.savefig("efficiency_frontier_comparison_Experiment2.png")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure DataFrame is ready
df = pd.DataFrame(results_table)

# Set up the figure with 3 subplots side-by-side
fig, axes = plt.subplots(1, 3, figsize=(24, 6))

# --- Plot 1: Perplexity (Lower is Better) ---
sns.lineplot(
    data=df, x='Threshold', y='PPL', hue='Method', style='Method',
    markers=True, markersize=10, linewidth=2.5, ax=axes[0], palette='viridis'
)
axes[0].set_title("Perplexity vs Restoration (Lower is Better)")
axes[0].set_xlabel("% Layers Restored to BF16")
axes[0].set_ylabel("Perplexity")
axes[0].grid(True, alpha=0.3)
axes[0].invert_yaxis() # Optional: If you want 'better' (lower) to be higher up, remove if standard view preferred.

# --- Plot 2: Accuracy (Higher is Better) ---
sns.lineplot(
    data=df, x='Threshold', y='Acc', hue='Method', style='Method',
    markers=True, markersize=10, linewidth=2.5, ax=axes[1], palette='magma'
)
axes[1].set_title("MMLU Accuracy vs Restoration (Higher is Better)")
axes[1].set_xlabel("% Layers Restored to BF16")
axes[1].set_ylabel("Accuracy")
axes[1].grid(True, alpha=0.3)

# --- Plot 3: Latency (Lower is Better) ---
sns.lineplot(
    data=df, x='Threshold', y='Latency', hue='Method', style='Method',
    markers=True, markersize=10, linewidth=2.5, ax=axes[2], palette='coolwarm'
)
axes[2].set_title("Inference Latency vs Restoration")
axes[2].set_xlabel("% Layers Restored to BF16")
axes[2].set_ylabel("Latency (Seconds)")
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("metrics_comparison_Experiment2.png")
plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import os
import shutil # Import shutil for copying files

# Define the directory to save files in Google Drive
save_dir = '/content/drive/MyDrive/Columbia-LLMSeminar/SLLM project/Jiayi/exp2'
os.makedirs(save_dir, exist_ok=True)

# Save results_table as CSV
df_results = pd.DataFrame(results_table)
df_results.to_csv(os.path.join(save_dir, 'exp2_results.csv'), index=False)
print(f"Results table saved to {os.path.join(save_dir, 'exp2_results.csv')}") # Corrected print statement

# Copy figures to Google Drive
figures_to_copy = [
    'efficiency_frontier_comparison_Experiment2.png',
    'metrics_comparison_Experiment2.png'
]

for fig_name in figures_to_copy:
    fig_path = os.path.join('/content', fig_name)
    if os.path.exists(fig_path):
        # Use shutil.copy2 to copy the file, then os.remove to delete the original
        shutil.copy2(fig_path, os.path.join(save_dir, fig_name))
        # os.remove(fig_path) # Remove original after copying
        print(f"Copied {fig_name} to {os.path.join(save_dir, fig_name)} and removed original.")
    else:
        print(f"Figure {fig_name} not found in current directory.")

# Experiment 3: Model Size
Research Question: Does this technique generalize to larger models? (Larger models are usually more robust to quantization; do they need less restoration?)

Fixed Variables:

Method: The "Winner" from Exp 2 (likely NF4 for speed or AWQ for accuracy).

Threshold: Fix to the "sweet spot" (e.g., 5%).

Independent Variable (Change this):

Model Size: 1.5B vs. 7B.